In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
%matplotlib inline
import matplotlib.pyplot as plt
import mglearn
import sklearn

# Naive Bayes 

    logistic과 linearSVC보다 훈련 속도가 빠르지만 일반화 성능이 좋지 않음.
    선형모델로는 학습이 오래 걸리는 '큰 데이터셋'에서는 쓸만하며 종종 사용된다고.

# Decision Tree

    정답에 가장 빨리 도달하는 yes / no 질문 목록을 학습한다는 뜻. 연속적인 데이터의 경우 'feature i는 값 a보다 큰가?' 와 같은 형태.